## 0. Setup

In [3]:
import os
import json
from pathlib import Path
from typing import List, Dict, Any
from datetime import datetime

# LangChain core
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

# Vector store
from langchain_chroma import Chroma

# Loaders
from langchain_community.document_loaders import PyPDFLoader, TextLoader, UnstructuredFileLoader

# OpenAI
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

ModuleNotFoundError: No module named 'langchain_chroma'

In [4]:
# Configuración de la API Key de OpenAI
import os

# 🔑 Opción 1: Defínela manualmente aquí (no recomendado en notebooks públicos)
# os.environ["OPENAI_API_KEY"] = "tu_api_key_aqui"

# 🔑 Opción 2 (recomendada): si estás en Colab, usa el gestor de secretos
try:
    from google.colab import userdata
    openai_api_key = userdata.get("OPENAI_API_KEY")
    os.environ["OPENAI_API_KEY"] = openai_api_key
except Exception as e:
    # Si no estás en Colab, intenta leer de variable local
    openai_api_key = os.getenv("OPENAI_API_KEY", "")
    if not openai_api_key:
        raise ValueError("❌ No se encontró OPENAI_API_KEY. Defínela antes de continuar.")

print("✅ API Key configurada correctamente")

✅ API Key configurada correctamente


## 1. Extracting files

In [ ]:
# Definir directorios de trabajo

# Carpeta donde estarán los documentos (definida en Celda 4 también)
DATA_DIR = Path("./data")
DATA_DIR.mkdir(exist_ok=True)

# Carpeta donde se guardará la base vectorial Chroma
PERSIST_DIR = Path("./data/chroma")
PERSIST_DIR.mkdir(exist_ok=True, parents=True)

print("✅ Directorios listos")
print("📂 DATA_DIR:", DATA_DIR.resolve())
print("📂 PERSIST_DIR:", PERSIST_DIR.resolve())

In [ ]:
import os
from pathlib import Path

# Carpeta donde deben estar tus documentos (PDF, DOCX, TXT, MD, HTML)
DATA_DIR = Path("./data")
DATA_DIR.mkdir(exist_ok=True)

print("✅ Entorno listo. Los documentos se tomarán desde:", DATA_DIR.resolve())

# Listar archivos encontrados en ./data
print("📦 Archivos detectados en ./data:")
for p in DATA_DIR.glob("*"):
    if p.suffix.lower() in [".pdf", ".docx", ".doc", ".txt", ".md", ".html", ".htm"]:
        print(" -", p.name)

In [ ]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader, TextLoader, UnstructuredFileLoader
from langchain.docstore.document import Document

folder_path = str(DATA_DIR)  # <-- usamos la carpeta definida en la celda 4

# Buscar archivos comunes (recursivo)
patterns = ["**/*.pdf", "**/*.txt", "**/*.md", "**/*.docx", "**/*.html", "**/*.htm"]
file_paths = []
for p in patterns:
    file_paths.extend(glob(os.path.join(folder_path, p), recursive=True))

if not file_paths:
    raise FileNotFoundError(
        f"No se encontraron archivos en {folder_path}. "
        "Coloca tus documentos en ./data."
    )

docs = []
for path in file_paths:
    try:
        low = path.lower()
        if low.endswith(".pdf"):
            loader = PyPDFLoader(path)
        elif low.endswith(".txt") or low.endswith(".md"):
            loader = TextLoader(path)
        else:
            loader = UnstructuredFileLoader(path)  # Para DOCX/HTML
        docs.extend(loader.load())
    except Exception as e:
        print(f"⚠️ Error cargando {os.path.basename(path)}: {e}")

print(f"✅ Se cargaron {len(docs)} fragmentos desde {len(file_paths)} archivos")

## 2. Text Splitting into Chunks

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Definir parámetros de fragmentación
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

# Dividir los documentos cargados en fragmentos
split_docs = text_splitter.split_documents(docs)

print(f"✅ Se crearon {len(split_docs)} fragmentos (chunk_size={CHUNK_SIZE}, overlap={CHUNK_OVERLAP})")

# Mostrar vista previa del primer fragmento
if split_docs:
    print("Ejemplo de fragmento:")
    print("Archivo:", split_docs[0].metadata.get("filename", "desconocido"))
    print("Texto:", split_docs[0].page_content[:200].replace("\n", " "), "...")

✅ Se crearon 49 chunks


## 3. Embedding

In [ ]:
from langchain_openai import OpenAIEmbeddings

# Crear embeddings con el modelo moderno (más rápido y económico que ada-002)
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=openai_api_key   # asegúrate que tu API Key está cargada en la celda 1
)

print("✅ Embeddings listos con text-embedding-3-small")

✅ Embeddings listos (dim=1536)


## 4. Vector Stores

In [ ]:
from langchain_chroma import Chroma
from pathlib import Path

# Carpeta donde se guardará la base vectorial
PERSIST_DIR = Path("./data/chroma")
PERSIST_DIR.mkdir(exist_ok=True, parents=True)

# Crear y persistir el vector store con los fragmentos y los embeddings ya definidos
vectordb = Chroma.from_documents(
    documents=split_docs,      # viene de la celda 8
    embedding=embeddings,      # viene de la celda 10
    persist_directory=str(PERSIST_DIR)
)

vectordb.persist()
print(f"✅ Vector store creado y guardado en: {PERSIST_DIR.resolve()}")

✅ Vector store creado y guardado en: /content/chroma_db


## 5. Retriving from the Persistant Vector Datastore

In [ ]:
from langchain_chroma import Chroma

# Reabrir Chroma desde disco usando los mismos embeddings
vectordb = Chroma(
    persist_directory=str(PERSIST_DIR),
    embedding_function=embeddings
)

# Preparar el retriever (k=4 por defecto)
retriever = vectordb.as_retriever(search_kwargs={"k": 4})

print("✅ Chroma reabierto desde:", PERSIST_DIR.resolve())
print("🔎 Retriever listo (k=4)")

✅ Store reabierto y retriever listo.
📁 Persist directory: /content/chroma_db
🔎 Parámetros de búsqueda: {'k': 4}


## 6. Retrivers in Langchain

In [ ]:
# 6) Retrieval con LLM (QA) y múltiples consultas — con citas y formato soporte
# ----------------------------------------------------------------------------
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from pathlib import Path

# LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=openai_api_key,  # ya definido en la celda 1
    temperature=0
)

# Prompt en español con formato de soporte
QA_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "Eres un asistente de soporte técnico. Responde en español usando SOLO el CONTEXTO.\n"
        "Formato exacto:\n"
        "**Diagnóstico breve**\n- …\n\n"
        "**Pasos para resolver**\n1. …\n2. …\n3. …\n\n"
        "**Verificación**\n- …\n\n"
        "**Notas / Advertencias**\n- …\n\n"
        "_Citas_: [archivo:página] [archivo:página]\n"
        "Si no hay evidencia suficiente en el CONTEXTO, dilo explícitamente.\n\n"
        "[CONTEXTO]\n{context}\n\n[PREGUNTA]\n{question}\n"
    ),
)

# Chain de QA con prompt y retorno de fuentes
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,                  # definido en celda 14
    return_source_documents=True,         # <-- importantísimo para citas
    chain_type="stuff",
    chain_type_kwargs={"prompt": QA_PROMPT}
)

# Utilidad para renderizar citas
def _cite(meta: dict) -> str:
    page = meta.get("page") or meta.get("page_number") or meta.get("source_page") or "?"
    fn = Path(meta.get("filename", meta.get("source","doc"))).name
    return f"[{fn}:{page}]"

# Lista de preguntas
queries = [
    "¿Qué información principal contienen estos documentos?",
    "¿La impresora atasca las hojas?"
]

# Ejecutar todas las consultas
for i, query in enumerate(queries, 1):
    result = qa.invoke({"query": query})
    answer = result.get("result", "")
    sources = result.get("source_documents", []) or []

    # Construir pie de citas si el modelo no las incluyó
    if "_Citas_:" not in answer and sources:
        cites = " ".join(_cite(d.metadata) for d in sources[:3])
        answer += f"\n\n_Citas_: {cites}"

    print(f"\n🔎 Pregunta {i}: {query}\n")
    print("🧠 Respuesta:\n", answer)


🔎 Pregunta 1: ¿Qué información principal contienen estos documentos?
🧠 Respuesta:
 Los documentos contienen información sobre procedimientos internos relacionados con la recepción, diagnóstico y reparación de equipos de cómputo. Incluyen secciones como "Inicio", "Nuevo Registro", "Editar Registro", "Cerrar Reparación", "Imprimir Reporte" y un anexo. También se menciona la revisión y emisión de un procedimiento específico con un código y la ubicación de la documentación en un módulo de control documental.

🔎 Pregunta 2: ¿La impresora atasca las hojas?
🧠 Respuesta:
 Sí, una de las fallas comunes es que el papel se traba siempre. La solución recomendada es llevar la impresora a mantenimiento al servicio técnico.
